# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [106]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

import json
from datetime import datetime as dt

In [107]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=user password=password")
cur = conn.cursor()

In [108]:
def get_files(filepath):
    '''
    Returns a list of all files with their respective paths of a given folder. 
    (Here it is used to collect all file paths in a dynamic way.)

            Parameters:
                    filepath: Path to folder where files are located

            Returns:
                    list of all files in a folder of a given path
    '''
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [109]:
#Reset dataframe and counter to zero

def read_files_to_df(file_elements):
    '''
    Returns a dataframe with all entries from files. 
    Prints out the number of files collected and read.
    (Here it is used to collect all entries about songs and songplays logged from single files
     and insert all of them in a dataframe.)

            Parameters:
                    file_elements: List of files (with path)

            Returns:
                    Dataframe with all single entries collected and read from the files
                    listed in the file_elements list
    '''
    df=None
    n=0

    for file in file_elements:
        #file content in JSON format is read from file
        data_element = pd.read_json(file, lines=True)
        #In 1st iteration (empty) Dataframe is generated
        if df is None:
            df=pd.DataFrame(data_element)
        #If the dataframe exists, in further iterations, 
        #each new line is appended to dataframe generated in 1st iteration
        else:
            #convert file entry to dataframe format
            new_df=pd.DataFrame(data_element)
            try:
                #reads contents of existing dataframe and adds the new dataframe with new entry
                frames = [df,new_df]
                df = pd.concat(frames)
            except:
                print('Error in file: '+str(n-1))
        n=n+1
    df.reset_index(drop=True, inplace=True)
    print('Number of files: '+str(n))
    return df

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [110]:
filepath = 'data/song_data'

In [111]:
def load_songfiles(filepath):
    '''
    Returns a dataframe with all songs found in song files. 
    First all (song) files in a given folder are read and their paths are added to a list.
    Then each file in this list is loaded, the contents is read and added to the final dataframe.

            Parameters:
                    file_path (path to folder with song files)

            Returns:
                    Dataframe with all single entries collected and read from the song files
                    identified and listed in the file list
    '''
    song_files = get_files(filepath)
    df_song = read_files_to_df(song_files)
    return df_song

In [112]:
#Dataframe with all songs is generated using LOAD_SONGFILES function
df_song=load_songfiles(filepath)
df_song.head()

Number of files: 73


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [113]:
#insert only first record
def insert_first_record_of_song_data(df):
    '''
    Returns first record of song dataframe. 

            Parameters:
                    df: Name of dataframe with songs

            Returns:
                    array of first song dataframe record
    '''
    song_data_cols = ['song_id','title','artist_id','year','duration']
    song_data = df[song_data_cols].values[0]
    return song_data

In [114]:
#insert all records
def insert_all_records_of_song_data(df):
    '''    
    Returns all records of song dataframe. 

            Parameters:
                    df: Name of dataframe with songs

            Returns:
                    array of all song dataframe records
    '''
    song_data_cols = ['song_id','title','artist_id','year','duration']
    song_data = df[song_data_cols].values
    return song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [115]:
def songs_df(df):
    '''    
    Extracts all song related columns and generates a new song only dataframe. 

            Parameters:
                    df: Name of dataframe with songs

            Returns:
                    Dataframe with extracted song related columns only based on songs dataframe
    '''
    songs_df = pd.DataFrame()
    song_data_cols = ['song_id','title','artist_id','year','duration']
    for column in song_data_cols:
        songs_df[column]=df[column]
    return songs_df

In [116]:
def insert_songs_df(df):
    '''    
    Iterates song only dataframe and inserts entry by entry to table songs using sql_queries.py

            Parameters:
                    df: Name of song data only dataframe

            Returns:
                    Nothing.
                    (Please check Postgres database if songs table has entries)
    '''
    #insert all records
    all_songs_df = songs_df(df_song)
    for i, row in all_songs_df.iterrows():
        cur.execute(song_table_insert, row)
        conn.commit()
    print("Number of rows in songs dataframe: " + str(i+1))
    return all_songs_df

In [117]:
#Inserts values of columns with song related information from dataframe with songs to table songs
all_songs_df=insert_songs_df(df_song)

Number of rows in songs dataframe: 73


In [118]:
all_songs_df.head()

,song_id,title,artist_id,year,duration
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
1,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
2,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
3,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
4,SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0,218.77506


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [119]:
def insert_first_record_of_artist_data(df):
    '''    
    Extracts artist related column values of first record and returns array with the read information. 

            Parameters:
                    df: Name of dataframe with songs where artist information can be found

            Returns:
                    Dataframe with extracted artist related columns only based on songs dataframe
    '''
    artist_data_cols = ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
    artist_data = df[artist_data_cols].values[0]
    return artist_data

In [120]:
def insert_all_records_of_artist_data(df):
    '''    
    Extracts artist related column values of all records and returns array with the read information. 

            Parameters:
                    df: Name of dataframe with songs where artist information can be fojnd

            Returns:
                    Dataframe with extracted artist related columns only based on songs dataframe
    '''
    artist_data_cols = ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
    artist_data = df[artist_data_cols].values
    return artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [121]:
def artists_df(df):
    '''    
    Extracts all artist related columns and generates a new artists only dataframe. 

            Parameters:
                    df: Name of dataframe with songs data read from files

            Returns:
                    Dataframe with extracted artist related columns only based on songs dataframe
    '''
    artists_df = pd.DataFrame()
    artist_data_cols = ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
    for column in artist_data_cols:
        artists_df[column]=df[column]
    return artists_df

In [122]:
def insert_artists_df(df):
    '''    
    Iterates artist only dataframe and inserts entry by entry to table artists using sql_queries.py

            Parameters:
                    df: Name of dataframe with songs data read from files

            Returns:
                    Nothing.
                    (Please check Postgres database if ARTISTS table has entries)
    '''
    #insert all records
    all_artists_df = artists_df(df)
    for i, row in all_artists_df.iterrows():
        cur.execute(artist_table_insert, row)
        conn.commit()
    print("Number of rows in artist dataframe: " + str(i+1))
    return all_artists_df

In [123]:
#Inserts values of columns with artist related information from dataframe with songs to table artists
all_artists_df=insert_artists_df(df_song)

Number of rows in artist dataframe: 73


In [124]:
all_artists_df.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
1,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
2,ARXR32B1187FB57099,Gob,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [125]:
filepath = 'data/log_data'

In [126]:
def load_logfiles(filepath):
    '''
    Returns a dataframe with all log data found in log files. 
    First all (log) files in a given folder are read and their paths are added to a list.
    Then each file in this list is loaded, the contents is read and added to the final dataframe.

            Parameters:
                    filepath (path to folder with log files)

            Returns:
                    Dataframe with all single entries collected and read from the log files
                    identified and listed in the file list
    '''
    log_files = get_files(filepath)
    df_log = read_files_to_df(log_files)
    return df_log

In [127]:
df=load_logfiles(filepath)
df.head()

Number of files: 30


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [128]:
def filter_df(df,col,parameter):
    '''
    Returns a dataframe filtered by a given parameter in a specific column col. 
            Parameters:
                    df: (dataframe to be filtered)
                    col: column in dataframe to be filtered
                    parameter: filter criterium
            Returns:
                    Dataframe with filtered entries
    '''
    df_cleaned=df.loc[df[col]==str(parameter)].copy()
    return df_cleaned

In [129]:
#user function FILTER_DF to filter dataframe with log data by 'NextSong' in column 'page'
df_cleaned=filter_df(df,'page','NextSong')
df_cleaned.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [130]:
#converts timestamps in milliseconds to datetime format
#returns lists with hour, day, week of year, month, year, and weekday 
def ts_converter_dataeng(df,column_labels):
    '''
    Returns dictionary of converted timestamps with elements and lists  
    with elements of converted timestamp in milliseconds in logfile (column ts)
            Parameters:
                    df: dataframe containing timestamp column
                    column_labels: list of names of columns of timestamp elements
            Returns:
                    Dictionary and lists 
    '''
    #Initialize all lists and dictionaries
    dict_time={}
    new_ts=[]
    new_ts_hour=[]
    new_ts_day=[]
    new_ts_week=[]
    new_ts_month=[]
    new_ts_year=[]
    new_ts_weekday=[]
    df_new=df['ts']
    #Append converted timestamp and elements
    for n in df_new:
        time_new_format=dt.fromtimestamp(n // 1000)
        new_ts.append(time_new_format)
        new_ts_hour.append(time_new_format.hour)
        new_ts_day.append(time_new_format.day)
        new_ts_week.append(time_new_format.isocalendar().week)
        new_ts_month.append(time_new_format.month)
        new_ts_year.append(time_new_format.year)
        new_ts_weekday.append(time_new_format.weekday())
    #Create Dictionaries and add lists
    dict_time[column_labels[0]]=new_ts
    dict_time[column_labels[1]]=new_ts_hour
    dict_time[column_labels[2]]=new_ts_day
    dict_time[column_labels[3]]=new_ts_week
    dict_time[column_labels[4]]=new_ts_month
    dict_time[column_labels[5]]=new_ts_year
    dict_time[column_labels[6]]=new_ts_weekday
    return dict_time,new_ts,new_ts_hour,new_ts_day,new_ts_week,new_ts_month,new_ts_year,new_ts_weekday

In [131]:
#generate dataframe from dictionary
def dict_to_df(dict):
    '''
    Returns dataframe of given dictionary
            Parameters:
                    dict: Dictionary that shall be converted to dataframe
            Returns:
                    t_df: Dataframe generated from dictionary
    '''
    dict_df=pd.DataFrame([dict])
    t_df=dict_df.apply(pd.Series.explode).reset_index()
    t_df.drop(['index'], axis=1)
    return t_df

In [132]:
#converts timestamps, generates dictionary and creates dataframe
def time_data_clean(df):
    '''
    Takes timestamp column of a given dataframe 
            Parameters:
                    df: Dataframe containing timestamp
            Returns:
                    time_df: Dataframe with converted timestamp and elements
    '''
    time_data = ()
    column_labels = ['timestamp','hour','day','week','month','year','weekday']
    dict_time,new_ts,new_ts_hour,new_ts_day,new_ts_week,new_ts_month,new_ts_year,new_ts_weekday=ts_converter_dataeng(df,column_labels)
    time_df = dict_to_df(dict_time)
    time_df = time_df.iloc[: , 1:]
    return time_df

In [133]:
#adds converted timestamp from time to log history
def add_conv_timestamp_to_df(df_orig, column_name_orig, df, column_name):
    '''
    Adds converted timestamp column to a given dataframe 
            Parameters:
                    df_orig: Dataframe containing original, non converted timestamp
                    df: New Dataframe where timestamp column shall be added
                    column_name_orig: Timestamp column name in original dataframe
                    column_name: Timestamp column name in new dataframe
            Returns:
                    Nothing
    '''
    df[column_name]=df_orig[column_name_orig]

In [134]:
#Generate dataframe with converted timestamps and add timestamp column to dataframe with log data
def create_time_df(df):
    '''
    Generates converted timestamp from timestamp column in UNIX format and adds it to a given dataframe 
            Parameters:
                    df: Dataframe where timestamp column shall be added
            Returns:
                    Nothing
    '''
    time_df=time_data_clean(df)
    add_conv_timestamp_to_df(time_df,'timestamp',df,'timestamp')

In [135]:
create_time_df(df)

In [156]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:33:56
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:36:10
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:40:34
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 05:34:01
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 05:36:13


In [137]:
#Show new time dataframe
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-11 03:33:56,3,11,45,11,2018,6
1,2018-11-11 03:36:10,3,11,45,11,2018,6
2,2018-11-11 03:40:34,3,11,45,11,2018,6
3,2018-11-11 05:34:01,5,11,45,11,2018,6
4,2018-11-11 05:36:13,5,11,45,11,2018,6


In [138]:
len(time_df)

8056

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [139]:
def insert_time_df(time_df):
    '''    
    Iterates time dataframe and inserts entry by entry to table time using sql_queries.py

            Parameters:
                    df: Name of dataframe with time data read from dataframe with log data

            Returns:
                    Nothing.
                    (Please check Postgres database if ARTISTS table has entries)
    '''
    #insert all records
    all_time_df = time_data_clean(df)
    add_conv_timestamp_to_df(all_time_df,'timestamp',df,'timestamp')
    for i, row in all_time_df.iterrows():
        cur.execute(time_table_insert, row)
        conn.commit()
    print("Number of rows in time dataframe: " + str(i+1))
    return all_time_df

In [148]:
#Inserts converted timestamp and time elements to Postgres SQL database
new_time_df=insert_time_df(df_cleaned)

Number of rows in time dataframe: 8056


In [149]:
new_time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-11 03:33:56,3,11,45,11,2018,6
1,2018-11-11 03:36:10,3,11,45,11,2018,6
2,2018-11-11 03:40:34,3,11,45,11,2018,6
3,2018-11-11 05:34:01,5,11,45,11,2018,6
4,2018-11-11 05:36:13,5,11,45,11,2018,6


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [150]:
def set_user_df(df):
    '''    
    Extracts all user related columns and generates a new user data only dataframe. 

            Parameters:
                    df: Name of dataframe with user data read from log files

            Returns:
                    Dataframe with extracted user data related columns only based on log information dataframe
    '''
    user_df = pd.DataFrame()
    user_data_cols = ['userId','firstName','lastName','gender','level']
    for column in user_data_cols:
        user_df[column]=df[column]
    return user_df

In [151]:
user_df = set_user_df(df_cleaned)

In [152]:
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [157]:
df_cleaned.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [153]:
def insert_user_df(user_df):
    '''    
    Iterates user dataframe and inserts entry by entry to table users using sql_queries.py

            Parameters:
                    user_df: Name of dataframe with user data read from dataframe with log data

            Returns:
                    Nothing.
                    (Please check Postgres database if USERS table has entries)
    '''
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)
        conn.commit()
    print("Number of rows in users dataframe: " + str(i+1))

In [1]:
insert_user_df(user_df)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [172]:
def insert_songplays(df,print_option):
    '''    
    Iterates dataframe and inserts entry by entry to table songplays
    given that artist_id in songs and artist_id in logs are equal using sql_queries.py

            Parameters:
                    df: Name of filtered dataframe with song data read from dataframe with songs data

            Returns:
                    Nothing.
                    (Please check Postgres database if SONGPLAYS table has entries)
    '''
    try:
        ct=0
        for index, row in df.iterrows():

            # get songid and artistid from song and artist tables
            cur.execute(song_select, (row.song, row.artist,str(row.length)))
            #print(row.song, row.artist, row.length)
            results = cur.fetchone()
            #print(results)

            if results:
                songid, artistid = results
                print("result/match for artist: "+str(row.artist))
                print("Song: "+str(row.song)+"\n" + "Artist: " + str(row.artist)+"\n" + "Listening time in s (length): " +str(row.length) +"\n" + str("Song ID: ") + str(songid)+"\n"+"Artist ID: "+str(artistid))
                songplay_data = (row.timestamp,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
                cur.execute(songplay_table_insert, songplay_data)
                conn.commit()
                ct=ct+1
            else:
                songid, artistid = None, None
                if print_option==True:
                    print("No results/match for artist: " + str(row.artist))
            #print("Number of results: "+str(ct))
            #insert songplay record
        print("Number of results: "+str(ct))
        print(str(ct)+ " dataset successfully inserted.")
    except:
        print("No data inserted, error in execution.")
    

In [176]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:33:56
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:36:10
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:40:34
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 05:34:01
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 05:36:13


In [173]:
insert_songplays(df,False)

result/match for artist: Elena
Song: Setanta matins
Artist: Elena
Listening time in s (length): 269.58322
Song ID: SOZCTXZ12AB0182364
Artist ID: AR5KOSW1187FB35FF4
Number of results: 1
1 dataset successfully inserted.


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [174]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.